In [1]:
import calendar

In [2]:
colors = {'NA': 'g', 'STG': '#1f77b4'}

In [ ]:
file = glob.glob(f"../Datasets/Dataset_NA_250.txt")[0]
NA_dataset =pd.read_csv(file, sep="\t", na_values='NaN')

NA_dates = NA_dataset["date_Argo"].str.split("/",expand=True).astype(int).rename(columns = {0:"Day",1:"Month",2:"Year"})
NA_dates["Year"] += 2000

In [ ]:


file = glob.glob(f"../Datasets/Dataset_NA_250.txt")[0]
NA_dataset =pd.read_csv(file, sep="\t", na_values='NaN')
NA_dates = NA_dataset["date_Argo"].str.split("/",expand=True).astype(int).rename(columns = {0:"Day",1:"Month",2:"Year"})
NA_dates["Year"] += 2000

# Count frequencies manually
NA_year_counts = NA_dates["Year"].value_counts().sort_index()
NA_month_counts = NA_dates["Month"].value_counts().sort_index()
# Convert month numbers to abbreviated names (e.g., 1 → Jan, 2 → Feb, ...)
month_labels = [calendar.month_abbr[m] for m in NA_month_counts.index]

file = glob.glob(f"../Datasets/Dataset_STG_250.txt")[0]
STG_dataset =pd.read_csv(file, sep="\t", na_values='NaN')


STG_dates = STG_dataset["date_Argo"].str.split("/",expand=True).astype(int).rename(columns = {0:"Day",1:"Month",2:"Year"})
STG_dates["Year"] += 2000

# Count frequencies manually
STG_year_counts = STG_dates["Year"].value_counts().sort_index()
STG_month_counts = STG_dates["Month"].value_counts().sort_index()

# --- Prepare month labels ---
month_labels = [calendar.month_abbr[m] for m in range(1, 13)]

# Ensure months 1–12 are always present for consistent comparison
for i in range(1, 13):
    if i not in NA_month_counts.index:
        NA_month_counts.loc[i] = 0
    if i not in STG_month_counts.index:
        STG_month_counts.loc[i] = 0

# Sort to align months correctly
NA_month_counts = NA_month_counts.sort_index()
STG_month_counts = STG_month_counts.sort_index()

# Get year union for alignment
all_years = sorted(set(NA_year_counts.index).union(set(STG_year_counts.index)))
NA_year_vals = [NA_year_counts.get(y, 0) for y in all_years]
STG_year_vals = [STG_year_counts.get(y, 0) for y in all_years]

# Bar positions
bar_width = 0.4
x_year = np.arange(len(all_years))
x_month = np.arange(12)

# --- Plot ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3), dpi=150)

# --- Plot 1: Year counts ---
ax1.bar(x_year - bar_width/2, NA_year_vals, width=bar_width, label='NA', color=colors["NA"], alpha=0.4, edgecolor='grey')
ax1.bar(x_year + bar_width/2, STG_year_vals, width=bar_width, label='STG', color=colors["STG"], alpha=0.4, edgecolor='grey')
ax1.set_xticks(x_year)
ax1.set_xticklabels(all_years)
ax1.set_ylabel("Frequency")
ax1.legend()

# --- Plot 2: Month counts ---
ax2.bar(x_month - bar_width/2, NA_month_counts.values, width=bar_width, label='NA', color=colors["NA"], alpha=0.4, edgecolor='grey')
ax2.bar(x_month + bar_width/2, STG_month_counts.values, width=bar_width, label='STG', color=colors["STG"], alpha=0.4, edgecolor='grey')
ax2.set_xticks(x_month)
ax2.set_xticklabels(month_labels, rotation = 45)


plt.tight_layout()

if export_fig_models:
    plt.savefig(f"../Results/Comparisons/Model_selection/Temporal_distribution.pdf")

plt.show()
